# Import required libraries

In [2]:
import dotenv
import openai
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

# Configure OpenAI settings

In [17]:
dotenv.load_dotenv()
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = os.getenv("AZURE_OPENAI_API_BASE")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
model = "text-embedding-ada-002"

# Configure vector store settings

In [18]:
vector_store_address = os.getenv("AZURE_COGNITIVE_SEARCH_URL")
vector_store_password = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")

# Create embeddings and vector store instances

In [21]:
embeddings = OpenAIEmbeddings(
    openai_api_key=openai.api_key,
    deployment=model,
    openai_api_type='azure',
    chunk_size=1
)
index_name = "acs-demo"
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

# Insert text and embeddings into vector store

In [22]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['OTQ5YjRkMjAtOTFkMi00YTVhLTk1OWMtMjg4NDk5YWJhOTQ0',
 'NTk0ZTRkM2MtZDVkNi00OGVmLWI5NTEtYzBhZjIzMDE0ODA1',
 'MDlmZDViMjAtNjY2OC00YWNiLTg0OGItN2U1ZThlNjk2MTg0',
 'MmM3NWY0ZDUtMzVlMS00MWNkLTlkZTctZDgyYjQ0NDJhNzNm',
 'ZjkyNmQwNTUtODhmZS00ZDUyLTgzMjItNWEyMzI5ZmQ3NDFm',
 'OTk4MGVlNzctNGM4YS00MjNhLWJhNGMtMjJlNTJlY2U3MjY0',
 'NWM2OWNjMzUtNTkyYi00NGVhLTg3NTktZmI0NGRlOTAzNzEx',
 'NDk1MzQxZjctZTUzNy00NWUwLThiZDctYjE5MTYxZjI3YTQw',
 'MTdkYjNiNTQtMjExOC00YzRiLWJjZTEtYzNhMjdiYmMxYzgy',
 'OThjOTY0NzQtNjNmNy00YWMzLThiZjctY2U3Yzc4ZmUyNTA0',
 'ODY0ODQyN2YtN2IwNy00YWViLWFhMmItOTRiOWE1ZDIxNWEy',
 'MWVkZDJiNTUtMjBjMy00MmQ0LWE0NjQtMzVjYjIwMDcxOWVk',
 'ZjhjYWMwYjgtOTQ0MS00ZmRhLWIyMTktMDIwNTcxYjJkZDQ0',
 'OTgyMzJiYjgtY2Q2Mi00NjBjLWJkZTMtNzgwYTliMWI1OTc1',
 'Nzc2ZDA1NjktMDEzNS00NTNhLTg3Y2QtYmNiNmEyZWU0YmQ1',
 'ZDcxZmUwOWMtOTAwOC00YzM2LTljMWQtMmNkYTM0NzJiOGM3',
 'M2FlZWQyMzctMWNjYi00NGY3LWI4NzktNWUzYjY4Yjk0ODk2',
 'Yjg1ODUxY2MtZTgxZC00ODY1LTljNjYtZjgzNWIwZWUwYWEz',
 'ZjE1MjQ2MDMtN2U0ZS00ZmU3LWFiNzEtOTk2YmMwOTc2

# Perform a vector similarity search

In [23]:
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
